In [48]:
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# 2.1 Produce the list of neighborhood names along with their URL's

In [49]:
url ='http://www.city-data.com/indexes/neighborhoods/FL/1/'
p = requests.get(url)
b = BeautifulSoup(p.content, "html.parser")

In [50]:
Hoods = b.find(class_ = 'index')
Hoods_links=Hoods.find_all('a')
#Hoods_links

In [51]:
links=[]
for link in Hoods_links:
    if link.get('href').__contains__('Tampa')==True:
        link='http://www.city-data.com'+link.get('href')
        links.append(link)
#links

In [52]:
Names=[]
for link in Hoods_links:
    if link.get('href').__contains__('Tampa')==True:
        x=link.get('href')
        x=x[14:-14]
        Names.append(x)
#Names

In [53]:
def Names_and_urls(url):
    p = requests.get(url)
    b = BeautifulSoup(p.content, "html.parser")
    column_names=['Name','Link']
    Links_Names=[]
    Hoods = b.find(class_ = 'index')
    Hoods_links=Hoods.find_all('a')
    
    for link in Hoods_links:
        if link.get('href').__contains__('Tampa')==True:
            x=link.get('href')
            x=x[14:-14]
            if link.get('href').__contains__('Tampa')==True:
                Link='http://www.city-data.com'+link.get('href')
                d = dict(zip(column_names,[x,Link]))
                Links_Names.append(d)
    return Links_Names

Names_and_urls('http://www.city-data.com/indexes/neighborhoods/FL/1/')

[{'Link': 'http://www.city-data.com/neighborhood/1000-Channelside-Condo-Tampa-FL.html',
  'Name': '1000-Channelside-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/1301-Soho-Condo-Tampa-FL.html',
  'Name': '1301-Soho-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/1508-South-Howard-Condo-Tampa-FL.html',
  'Name': '1508-South-Howard-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/1611-6th-Avenue-Townhomes-Tampa-FL.html',
  'Name': '1611-6th-Avenue-Townhomes'},
 {'Link': 'http://www.city-data.com/neighborhood/2002-Dekle-Condo-Tampa-FL.html',
  'Name': '2002-Dekle-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/215-Verne-Condo-Tampa-FL.html',
  'Name': '215-Verne-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/2513-Maryland-Avenue-Condo-Tampa-FL.html',
  'Name': '2513-Maryland-Avenue-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/2515-Maryland-Avenue-Condo-Tampa-FL.html',
  'Name': '2515-Maryland-Avenue-Condo'},
 {'Link': 'http:

In [14]:
url = 'http://www.city-data.com/indexes/neighborhoods/FL/%d/'

l = []
for page in range(1,134):
    print(url%page)
    l = l + Names_and_urls(url%page)
    time.sleep(2)
    

http://www.city-data.com/indexes/neighborhoods/FL/1/
http://www.city-data.com/indexes/neighborhoods/FL/2/
http://www.city-data.com/indexes/neighborhoods/FL/3/
http://www.city-data.com/indexes/neighborhoods/FL/4/
http://www.city-data.com/indexes/neighborhoods/FL/5/
http://www.city-data.com/indexes/neighborhoods/FL/6/
http://www.city-data.com/indexes/neighborhoods/FL/7/
http://www.city-data.com/indexes/neighborhoods/FL/8/
http://www.city-data.com/indexes/neighborhoods/FL/9/
http://www.city-data.com/indexes/neighborhoods/FL/10/
http://www.city-data.com/indexes/neighborhoods/FL/11/
http://www.city-data.com/indexes/neighborhoods/FL/12/
http://www.city-data.com/indexes/neighborhoods/FL/13/
http://www.city-data.com/indexes/neighborhoods/FL/14/
http://www.city-data.com/indexes/neighborhoods/FL/15/
http://www.city-data.com/indexes/neighborhoods/FL/16/
http://www.city-data.com/indexes/neighborhoods/FL/17/
http://www.city-data.com/indexes/neighborhoods/FL/18/
http://www.city-data.com/indexes/neig

In [15]:
l

[{'Link': 'http://www.city-data.com/neighborhood/1000-Channelside-Condo-Tampa-FL.html',
  'Name': '1000-Channelside-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/1301-Soho-Condo-Tampa-FL.html',
  'Name': '1301-Soho-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/1508-South-Howard-Condo-Tampa-FL.html',
  'Name': '1508-South-Howard-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/1611-6th-Avenue-Townhomes-Tampa-FL.html',
  'Name': '1611-6th-Avenue-Townhomes'},
 {'Link': 'http://www.city-data.com/neighborhood/2002-Dekle-Condo-Tampa-FL.html',
  'Name': '2002-Dekle-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/215-Verne-Condo-Tampa-FL.html',
  'Name': '215-Verne-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/2513-Maryland-Avenue-Condo-Tampa-FL.html',
  'Name': '2513-Maryland-Avenue-Condo'},
 {'Link': 'http://www.city-data.com/neighborhood/2515-Maryland-Avenue-Condo-Tampa-FL.html',
  'Name': '2515-Maryland-Avenue-Condo'},
 {'Link': 'http:

In [16]:
df=pd.DataFrame(l)
df.set_index('Name', inplace=True)
df.to_csv('LinksandNames.csv')
df.shape

(2393, 1)

In [17]:
%cat LinksandNames.csv | head -10

Name,Link
1000-Channelside-Condo,http://www.city-data.com/neighborhood/1000-Channelside-Condo-Tampa-FL.html
1301-Soho-Condo,http://www.city-data.com/neighborhood/1301-Soho-Condo-Tampa-FL.html
1508-South-Howard-Condo,http://www.city-data.com/neighborhood/1508-South-Howard-Condo-Tampa-FL.html
1611-6th-Avenue-Townhomes,http://www.city-data.com/neighborhood/1611-6th-Avenue-Townhomes-Tampa-FL.html
2002-Dekle-Condo,http://www.city-data.com/neighborhood/2002-Dekle-Condo-Tampa-FL.html
215-Verne-Condo,http://www.city-data.com/neighborhood/215-Verne-Condo-Tampa-FL.html
2513-Maryland-Avenue-Condo,http://www.city-data.com/neighborhood/2513-Maryland-Avenue-Condo-Tampa-FL.html
2515-Maryland-Avenue-Condo,http://www.city-data.com/neighborhood/2515-Maryland-Avenue-Condo-Tampa-FL.html
2518-W-Kansas-Avenue-Condo,http://www.city-data.com/neighborhood/2518-W-Kansas-Avenue-Condo-Tampa-FL.html
cat: write error: Broken pipe


# 2.2 Scrape neighborhood data based on URL

In [ ]:
# make a funciton to scrape the data from one url
#return a dictionary
#{'name':"neighname",
# "population":"555"}
#Try excepts if the neigh doesnt even have twenty fields.

In [56]:
url ='http://www.city-data.com/neighborhood/Alice-Kelly-s-Tampa-FL.html'
p = requests.get(url)
b = BeautifulSoup(p.content, "html.parser")

In [72]:
first=b.find(class_='row')
second=first.find(class_='content-item').text
m=second.split('\n')
m[0:-1]

['Area: 0.043 square miles',
 'Population: 44',
 "Population density:Alice Kelly's:1,020 people per square mileTampa:3,366 people per square mile"]

In [87]:
first=b.find(class_='row')
second=first.find_all(class_='content-item')
third=first.find_all(class_='hgraph')
third[0]

<div class="hgraph"><table><tr><td><b>Alice Kelly's:</b></td><td><p class="h" style="padding-left:45px;"></p>1,020 <b>people per square mile</b></td></tr><tr><td><b>Tampa:</b></td><td><p class="a" style="padding-left:150px;"></p>3,366 <b>people per square mile</b></td></tr></table></div>

In [107]:
third[3].find_all('tr')[0].find_all('td')[1].text

'21'

In [110]:
def parse_neigh(third, ind):
    return third[ind].find_all('tr')[0].find_all('td')[1].text

In [112]:
values = [parse_neigh(third, ind) for ind in range(0,28)]
keys = ['Population density',
        'Median income',
        'Median Rent',
        'Population of males'
        'Median age of males',
        'Median age of memales',
        'Average household size',
        'Percentage of mamily households',
        'Percentage of married-couple families',
        'Percentage of married-couple families with children',
        'Percentage of ingle motehr households',
        'Percentage of never Married Males Over 15',
        'Percentage of never Married Females Over 15',
        'Percentage of people that do not speak english',
        'Percentage of people born in the state',
        'Percentage of people born in another state',
        'Percentage of native residents born outside the US',
        '',
        '',
        '',
        '',
        '',
        '',
        '',
        '']
d = dict()

['1,020 people per square mile',
 '$52,778',
 '$911',
 '21',
 '35.2 years',
 '29.3 years',
 '1.3 people',
 '20.7%',
 '10.7%',
 '56.5%',
 '102.7%',
 '32.0%',
 '34.9%',
 '0.0%',
 '28.0%',
 '55.9%',
 '5.3%',
 '10.7%',
 '1.6',
 '1.2',
 '36.8%',
 '242.5',
 '71.1%',
 '40.5%',
 '19.2%',
 '11.8%',
 '6.6%',
 '21.3%']

In [57]:
cont=[]
for el in b.find(class_='row'):
    for item in el.find_all(class_='content-item'):
        x=item.text
        cont.append(x)
        #print(x)
    for item in el.find_all(class_='hgraph'):
        y=item.text
        cont.append(y)
        #print(y)

In [85]:
cont2=[]
for item in cont:
    x=item.split('\n')
    x=x[0:-1]
    cont2.append(x)
print(cont2[0:20])

[['Area: 0.043 square miles', 'Population: 44', "Population density:Alice Kelly's:1,020 people per square mileTampa:3,366 people per square mile"], ['Median household income in 2016: ', "Alice Kelly's:$52,778Tampa:$50,405", ''], ['Median rent in 2016:', "Alice Kelly's:$911Tampa:$910", ''], ['Zip codes: 33606. '], ['Males:21Females:23', 'Median age:Males:This neighborhood:35.2 yearsWhole city:35.4 years', 'Females:This neighborhood:29.3 yearsWhole city:36.7 years'], ['', "Average household size:Alice Kelly's:1.3 peopleTampa:2.4 people", '', "Percentage of family households:Alice Kelly's:20.7%Tampa:40.4%", 'Percentage of married-couple families (among all households):', 'Here:10.7%Tampa:33.9%', '', 'Percentage of married-couple families with children (among all households):', "Alice Kelly's:56.5%Tampa:21.7%", '', 'Percentage of single-mother households (among all households):', "Alice Kelly's:102.7%Tampa:14.4%", ''], [], ['Percentage of never married males 15 years old and over:', "Alice

# 2.3 Scraping all the neighborhoods

In [59]:
links2=[]
for item in l:
    x=item
    for item in x.values():
        links2.append(item)
links3=links2[1::2]

In [60]:
cont3=[]
def neighborhood_scraper(url):
    cont=[]
    cont2=[]
    p = requests.get(url)
    b = BeautifulSoup(p.content, "html.parser")
    for el in b.find(class_='row'):
        for item in el.find_all(class_='content-item'):
            x=item.text
            cont.append(x)
        for item in el.find_all(class_='hgraph'):
            y=item.text
            cont.append(y)
    for item in cont:
        z=item.split('\n')
        cont2.append(z)
    cont3.append(cont2[0:20])
    
    
neighborhood_scraper('http://www.city-data.com/neighborhood/Alice-Kelly-s-Tampa-FL.html')
cont3

[[['Area: 0.043 square miles',
   'Population: 44',
   "Population density:Alice Kelly's:1,020 people per square mileTampa:3,366 people per square mile",
   ''],
  ['Median household income in 2016: ',
   "Alice Kelly's:$52,778Tampa:$50,405",
   '',
   ''],
  ['Median rent in 2016:', "Alice Kelly's:$911Tampa:$910", '', ''],
  ['Zip codes: 33606. ', ''],
  ['Males:21Females:23',
   'Median age:Males:This neighborhood:35.2 yearsWhole city:35.4 years',
   'Females:This neighborhood:29.3 yearsWhole city:36.7 years',
   ''],
  ['',
   "Average household size:Alice Kelly's:1.3 peopleTampa:2.4 people",
   '',
   "Percentage of family households:Alice Kelly's:20.7%Tampa:40.4%",
   'Percentage of married-couple families (among all households):',
   'Here:10.7%Tampa:33.9%',
   '',
   'Percentage of married-couple families with children (among all households):',
   "Alice Kelly's:56.5%Tampa:21.7%",
   '',
   'Percentage of single-mother households (among all households):',
   "Alice Kelly's:102.7

In [ ]:
cont3=[]
for link in links3:
    neighborhood_scraper(link)
    time.sleep(5)
    print('done')
cont3